### 🔗 Training Notebook Source

**The model used for this inference was trained and developed in the following Kaggle Notebook.**

[**View the full Training Notebook here**](https://www.kaggle.com/code/tasmim/train-csiro-image2biomass-prediction?scriptVersionId=272144434)


In [ ]:
# ============================================================================
# CSIRO Image2Biomass Prediction - INFERENCE ONLY
# ============================================================================

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import warnings
import os
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
class CFG:
    # Paths
    test_csv = '/kaggle/input/csiro-biomass/test.csv'
    test_dir = '/kaggle/input/csiro-biomass/test'
    model_dir = '/kaggle/input/csiro-models'  # Directory containing model checkpoints
    output_file = 'submission.csv'
    
    # Model settings (MUST match training configuration)
    model_name = 'tf_efficientnetv2_m'
    img_size = 512
    n_folds = 5  # Number of folds to ensemble
    
    # Inference settings
    batch_size = 32  # Can be larger than training since no gradients
    num_workers = 4
    use_tta = True  # Test-Time Augmentation
    tta_steps = 10   # Number of TTA augmentations
    
    # Target names (order matters!)
    targets = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
print(f"Using device: {CFG.device}")
print(f"Will ensemble {CFG.n_folds} models")
print(f"TTA enabled: {CFG.use_tta} ({CFG.tta_steps} augmentations)" if CFG.use_tta else "TTA disabled")

# ============================================================================
# DATASET CLASS
# ============================================================================
class BiomassTestDataset(Dataset):
    """
    Dataset for test/inference data.
    Loads images and applies tabular feature scaling.
    Note: Test data may not have metadata features (NDVI, Height).
    """
    def __init__(self, df, img_dir, transform=None, tabular_scaler=None, has_metadata=True):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.has_metadata = has_metadata
        
        # Prepare tabular features
        if has_metadata:
            # Use actual metadata from CSV
            tabular_data = df[['Pre_GSHH_NDVI', 'Height_Ave_cm']].fillna(0).values
            
            if tabular_scaler is not None:
                self.tabular_features = tabular_scaler.transform(tabular_data)
            else:
                self.tabular_features = tabular_data
        else:
            # No metadata available - use zeros (scaled mean)
            # This is reasonable since StandardScaler centers data around 0
            print("  ⚠ No metadata in test set - using zero values (scaled mean)")
            self.tabular_features = np.zeros((len(df), 2), dtype=np.float32)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        img_path = f"{self.img_dir}/{row['image_path'].split('/')[-1]}"
        image = cv2.imread(img_path)
        
        if image is None:
            raise ValueError(f"Failed to load image: {img_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        # Get tabular features
        tabular = torch.tensor(self.tabular_features[idx], dtype=torch.float32)
        
        return image, tabular

# ============================================================================
# AUGMENTATION TRANSFORMS
# ============================================================================
def get_inference_transforms():
    """Standard transforms for inference (no augmentation)"""
    return A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_tta_transforms():    
    """
    Test-Time Augmentation transforms.
    Returns list of diverse augmentation pipelines (10 steps).
    """
    base = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    return [
        # 1. Base (No TTA)
        base,
        # 2. Horizontal flip
        A.Compose([A.HorizontalFlip(p=1.0), base]),
        # 3. Vertical flip
        A.Compose([A.VerticalFlip(p=1.0), base]),
        # 4. Rotate 90 degrees
        A.Compose([A.Rotate(limit=(90, 90), p=1.0, border_mode=cv2.BORDER_REFLECT_101), base]),
        # 5. Rotate 270 degrees
        A.Compose([A.Rotate(limit=(-90, -90), p=1.0, border_mode=cv2.BORDER_REFLECT_101), base]),
        
        # 6. Slight rotation and shift
        A.Compose([
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0, rotate_limit=10, p=1.0, border_mode=cv2.BORDER_REFLECT_101),
            base
        ]),
        # 7. Brightness variation
        A.Compose([
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=1.0),
            base
        ]),
        # 8. Hue, Saturation, Value variation
        A.Compose([
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=15, p=1.0),
            base
        ]),
        # 9. Mild Perspective Distortion
        A.Compose([
            A.Perspective(scale=(0.02, 0.05), p=1.0, keep_size=True, fit_output=True),
            base
        ]),
        # 10. Gaussian Blur (for texture generalization)
        A.Compose([
            A.GaussianBlur(blur_limit=(3, 5), p=1.0),
            base
        ]),
    ][:CFG.tta_steps] # Use slicing to respect CFG.tta_steps
# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================
class BiomassModel(nn.Module):
    """
    Multi-modal model combining image and tabular features.
    MUST match the architecture used during training!
    """
    def __init__(self, model_name, pretrained=False):
        super(BiomassModel, self).__init__()
        
        # Image encoder (EfficientNet backbone)
        self.backbone = timm.create_model(
            model_name, 
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
        )
        
        # Get feature dimension
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, CFG.img_size, CFG.img_size)
            img_features = self.backbone(dummy_input).shape[1]
        
        # Tabular feature encoder
        self.tabular_encoder = nn.Sequential(
            nn.Linear(2, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        # Fusion layer
        fusion_dim = img_features + 128
        self.fusion = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        # Output heads (5 targets)
        self.head_green = nn.Linear(256, 1)
        self.head_dead = nn.Linear(256, 1)
        self.head_clover = nn.Linear(256, 1)
        self.head_gdm = nn.Linear(256, 1)
        self.head_total = nn.Linear(256, 1)
    
    def forward(self, image, tabular):
        # Extract features
        img_features = self.backbone(image)
        tab_features = self.tabular_encoder(tabular)
        
        # Fuse
        combined = torch.cat([img_features, tab_features], dim=1)
        fused = self.fusion(combined)
        
        # Predict
        out_green = self.head_green(fused)
        out_dead = self.head_dead(fused)
        out_clover = self.head_clover(fused)
        out_gdm = self.head_gdm(fused)
        out_total = self.head_total(fused)
        
        outputs = torch.cat([out_green, out_dead, out_clover, out_gdm, out_total], dim=1)
        return outputs

# ============================================================================
# INFERENCE FUNCTIONS
# ============================================================================
def predict_single_tta(model, dataset, device, tta_transform, has_metadata):
    """
    Make predictions with a single TTA transform.
    """
    # Create dataset with specific TTA transform
    dataset_tta = BiomassTestDataset(
        dataset.df, 
        dataset.img_dir,
        transform=tta_transform,
        tabular_scaler=None,  # Already scaled in original dataset
        has_metadata=has_metadata
    )
    dataset_tta.tabular_features = dataset.tabular_features  # Use same tabular features
    
    loader = DataLoader(
        dataset_tta, 
        batch_size=CFG.batch_size,
        shuffle=False, 
        num_workers=CFG.num_workers,
        pin_memory=True
    )
    
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for images, tabular in loader:
            images = images.to(device)
            tabular = tabular.to(device)
            
            outputs = model(images, tabular)
            predictions.append(outputs.cpu().numpy())
    
    return np.vstack(predictions)

def predict_with_tta(model, dataset, device, has_metadata):
    """
    Make predictions with Test-Time Augmentation.
    Averages predictions across multiple augmentations.
    """
    if not CFG.use_tta:
        # No TTA - single prediction
        tta_transforms = [get_inference_transforms()]
    else:
        # Multiple TTA transforms
        tta_transforms = get_tta_transforms()[:CFG.tta_steps]
    
    print(f"  Making predictions with {len(tta_transforms)} TTA variations...")
    
    all_tta_preds = []
    for tta_idx, tta_transform in enumerate(tta_transforms):
        preds = predict_single_tta(model, dataset, device, tta_transform, has_metadata)
        all_tta_preds.append(preds)
        print(f"    TTA {tta_idx + 1}/{len(tta_transforms)} complete")
    
    # Average across TTA augmentations
    avg_preds = np.mean(all_tta_preds, axis=0)
    return avg_preds

def load_model_checkpoint(fold):
    """Load model checkpoint with all necessary components"""
    checkpoint_path = os.path.join(CFG.model_dir, f'best_model_fold{fold}.pth')
    
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Model checkpoint not found: {checkpoint_path}")
    
    # Load with weights_only=False to allow loading sklearn scalers
    # This is safe since we trust our own checkpoints
    checkpoint = torch.load(checkpoint_path, map_location=CFG.device, weights_only=False)
    
    # Handle different checkpoint formats
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        # New format with scalers
        model_state = checkpoint['model_state_dict']
        tabular_scaler = checkpoint.get('tabular_scaler', None)
        target_scaler = checkpoint.get('target_scaler', None)
    else:
        # Old format (just model weights)
        model_state = checkpoint
        tabular_scaler = None
        target_scaler = None
        print(f"  Warning: Fold {fold} checkpoint doesn't contain scalers")
    
    return model_state, tabular_scaler, target_scaler

# ============================================================================
# MAIN INFERENCE PIPELINE
# ============================================================================
def main():
    print("="*70)
    print("CSIRO BIOMASS PREDICTION - INFERENCE")
    print("="*70)
    
    # 1. Load test data
    print("\n[1/5] Loading test data...")
    test_df = pd.read_csv(CFG.test_csv)
    print(f"✓ Loaded {len(test_df)} test samples")
    print(f"✓ Columns in test data: {test_df.columns.tolist()}")
    
    # Get unique images (test.csv has 5 rows per image, one per target)
    test_df_unique = test_df.drop_duplicates(subset=['image_path']).reset_index(drop=True)
    print(f"✓ Found {len(test_df_unique)} unique images")
    
    # Check if metadata features are available
    has_metadata = 'Pre_GSHH_NDVI' in test_df_unique.columns and 'Height_Ave_cm' in test_df_unique.columns
    
    if has_metadata:
        print(f"✓ Test data has metadata features (NDVI, Height)")
    else:
        print(f"⚠ Test data does NOT have metadata features")
        print(f"  Will use zero values (scaled mean) for tabular features")
        # Add dummy columns so dataset creation doesn't fail
        test_df_unique['Pre_GSHH_NDVI'] = 0.0
        test_df_unique['Height_Ave_cm'] = 0.0
    
    # 2. Verify model checkpoints exist
    print("\n[2/5] Checking model checkpoints...")
    FOLD_VALIDATION_SCORES = [0.6078, 0.6534, 0.7948, 0.7265, 0.7791]
    available_folds = []
    for fold in range(CFG.n_folds):
        checkpoint_path = os.path.join(CFG.model_dir, f'best_model_fold{fold}.pth')
        if os.path.exists(checkpoint_path):
            available_folds.append(fold)
            print(f"✓ Found checkpoint for fold {fold}")
        else:
            print(f"✗ Missing checkpoint for fold {fold}")
    
    if len(available_folds) == 0:
        raise FileNotFoundError("No model checkpoints found! Train models first.")
    
    print(f"\n✓ Will use {len(available_folds)} models for ensemble")
    
    # 3. Generate predictions from each fold
    print("\n[3/5] Generating predictions...")
    all_fold_predictions = []
    
    for fold in available_folds:
        print(f"\n--- Fold {fold + 1}/{CFG.n_folds} ---")
        
        # Load checkpoint
        model_state, tabular_scaler, target_scaler = load_model_checkpoint(fold)
        
        # Create model and load weights
        model = BiomassModel(CFG.model_name, pretrained=False).to(CFG.device)
        model.load_state_dict(model_state)
        model.eval()
        print(f"✓ Model loaded successfully")
        
        # Create dataset
        test_dataset = BiomassTestDataset(
            test_df_unique,
            CFG.test_dir,
            transform=get_inference_transforms(),
            tabular_scaler=tabular_scaler,
            has_metadata=has_metadata
        )
        
        # Make predictions with TTA
        fold_predictions = predict_with_tta(model, test_dataset, CFG.device, has_metadata)
        
        # Inverse transform if target scaler exists
        if target_scaler is not None:
            fold_predictions = target_scaler.inverse_transform(fold_predictions)
            print(f"✓ Predictions scaled back to original range")
        
        all_fold_predictions.append(fold_predictions)
        print(f"✓ Fold {fold} predictions: shape {fold_predictions.shape}")
        
        # Print sample predictions
        print(f"  Sample prediction: {fold_predictions[0]}")
        
        # Clean up
        del model
        torch.cuda.empty_cache()
    
    # 4. Ensemble predictions
    print("\n[4/5] Ensembling predictions from all folds...")
    if len(available_folds) < 2 or all(score == 0.0 for score in FOLD_VALIDATION_SCORES):
        print("⚠ Simple averaging due to insufficient folds or missing scores.")
        final_predictions = np.mean(all_fold_predictions, axis=0)
    else:
        # --- WEIGHTED ENSEMBLE LOGIC ---
        print(f"✓ Using Weighted Ensemble based on R² scores: {FOLD_VALIDATION_SCORES}")
        
        # 1. Filter scores to only include available folds
        available_scores = [FOLD_VALIDATION_SCORES[fold] for fold in available_folds]
        
        # 2. Normalize weights (must be non-negative)
        weights = np.array([max(0.0, score) for score in available_scores])
        weights = weights / weights.sum()
        
        print(f"Normalized Weights: {weights}")
        
        # 3. Apply weights to predictions
        final_predictions = np.sum(
            np.stack(all_fold_predictions, axis=0) * weights[:, np.newaxis, np.newaxis], 
            axis=0
        )

    
    # 5. Create submission file
    print("\n[5/5] Creating submission file...")
    submission_rows = []
    
    for idx, row in test_df_unique.iterrows():
        # Extract image ID from path
        image_id = row['image_path'].split('/')[-1].replace('.jpg', '')
        
        # Create one row per target
        for target_idx, target_name in enumerate(CFG.targets):
            sample_id = f"{image_id}__{target_name}"
            
            # Get prediction and ensure non-negative
            prediction = max(0.0, final_predictions[idx, target_idx])
            
            submission_rows.append({
                'sample_id': sample_id,
                'target': prediction
            })
    
    # Create DataFrame and save
    submission_df = pd.DataFrame(submission_rows)
    submission_df.to_csv(CFG.output_file, index=False)
    
    print(f"\n{'='*70}")
    print(f"✓ INFERENCE COMPLETE!")
    print(f"{'='*70}")
    print(f"✓ Submission file saved: {CFG.output_file}")
    print(f"✓ Total predictions: {len(submission_df)}")
    print(f"✓ Expected format: sample_id, target")
    print(f"\nFirst few rows:")
    print(submission_df.head(10))
    print(f"\nLast few rows:")
    print(submission_df.tail(5))
    
    # Validation checks
    print(f"\n--- Submission Validation ---")
    expected_rows = len(test_df_unique) * 5  # 5 targets per image
    if len(submission_df) == expected_rows:
        print(f"✓ Row count correct: {len(submission_df)} rows")
    else:
        print(f"⚠ Warning: Expected {expected_rows} rows, got {len(submission_df)}")
    
    # Check for any NaN or negative values
    if submission_df['target'].isna().sum() > 0:
        print(f"⚠ Warning: {submission_df['target'].isna().sum()} NaN values found")
    else:
        print(f"✓ No NaN values")
    
    if (submission_df['target'] < 0).sum() > 0:
        print(f"⚠ Warning: {(submission_df['target'] < 0).sum()} negative values found")
    else:
        print(f"✓ No negative values")
    
    print(f"\n✓ Ready for submission!")

if __name__ == '__main__':
    main()


In [ ]:
sub = pd.read_csv('/kaggle/working/submission.csv')
sub.head()